In [1]:
import os
import pandas as pd
import numpy as np
import flydataclass as fc
from scipy.stats import chi2_contingency
from itertools import product
import seaborn as sns
import matplotlib.pyplot as plt

def convert_pvalue_to_asterisks(pvalue):
    if pvalue <= 0.0001:
        return "****"
    elif pvalue <= 0.001:
        return "***"
    elif pvalue <= 0.01:
        return "**"
    elif pvalue <= 0.05:
        return "*"
    return "ns"


f1_dir = "./data/f1"
f2_dir = "./data/f2"

# Load data
* F1

In [2]:
csv_files = [
    file for file in os.listdir(f1_dir) if file.endswith('.csv')
]

f1_data = []
for file in csv_files:
    if file == "data_全体汇总.csv":
        continue
    file_path = os.path.join(f1_dir, file)
    df = pd.read_csv(file_path)
    numpy_df = df.to_numpy()
    for row in numpy_df[2:, ]:
        f1_data.append(fc.GroupData(row, 1))

* F2

In [3]:
csv_files = []
for item in os.listdir(f2_dir):
    item_path = os.path.join(f2_dir, item)
    if os.path.isdir(item_path):
        for file in os.listdir(item_path):
            if file.endswith(".csv"):
                filr_path = os.path.join(item_path, file)
                csv_files.append(filr_path)

f2_data = []
for file in csv_files:
    if file == "./data/f2\\all data\\合计.csv":
        continue
    df = pd.read_csv(file)
    df = df.iloc[2:10, 2:6].join(df.iloc[2:10, 9:13])
    numpy_df = df.to_numpy()
    f2_data.append(fc.GroupData(numpy_df, 2))

# filter data

In [4]:
f1_orth_valid = 0
f1_comp_valid = 0
f2_valid = 0

for group_data in f1_data:
    if group_data.filter_data("orth"):
        f1_orth_valid = f1_orth_valid + 1
    if group_data.filter_data("comp"):
        f1_comp_valid = f1_comp_valid + 1
print("f1_orth_valid:", f1_orth_valid)
print("f1_comp_valid:", f1_comp_valid)

for group_data in f2_data:
    if group_data.filter_data():
        f2_valid = f2_valid + 1
print("f2_both_valid:", f2_valid)

f1_orth_valid: 73
f1_comp_valid: 91
f2_both_valid: 64


# data analysis
1.F1代中正交反交分别对数据进行测算并卡平方检验。理想数据是应该得到的各基因型比例与所拿到的果蝇总数相乘之积，再与实际各表型数据进行检验。
* 获取各表型数据

In [5]:
print("f1代正交数据")
orth_sum = 0
for group in f1_data:
    orth_sum = group.get_count("orth") + orth_sum
    
print("* 正交数据总数：", orth_sum)
print("* 正交数据各表型数据：")
orth_grey_body = orth_black_body = 0
orth_red_eye = orth_white_eye = 0
orth_long_wing = orth_short_wing = 0
orth_straight_bristle = orth_curly_bristle = 0
orth_female = orth_male = 0
f1_orth_female = f1_orth_male = 0 # 灰红直长雌, 灰白焦小雄
for group in f1_data:
    orth_grey_body = orth_grey_body + group.get_body_color_count("grey", "orth")
    orth_black_body = orth_black_body + group.get_body_color_count("black", "orth")
    orth_red_eye = orth_red_eye + group.get_eye_color_count("red", "orth")
    orth_white_eye = orth_white_eye + group.get_eye_color_count("white", "orth")
    orth_long_wing = orth_long_wing + group.get_wing_shape_count("long", "orth")
    orth_short_wing = orth_short_wing + group.get_wing_shape_count("short", "orth")
    orth_straight_bristle = orth_straight_bristle + group.get_bristle_type_count("straight", "orth")
    orth_curly_bristle = orth_curly_bristle + group.get_bristle_type_count("curly", "orth")
    orth_female = orth_female + group.get_gender_count("female", "orth")
    orth_male = orth_male + group.get_gender_count("male", "orth")
    f1_orth_female = f1_orth_female + group.get_specific_count([1, 1, 1, 1, 1], "orth")
    f1_orth_male = f1_orth_male + group.get_specific_count([1, 0, 0, 0, 0], "orth")
    
print("     体色: 灰", orth_grey_body, ", 黑", orth_black_body)
print("     眼色: 红", orth_red_eye, ", 白", orth_white_eye)
print("     翅型: 长", orth_long_wing, ", 短", orth_short_wing)
print("     刚毛: 直", orth_straight_bristle, ", 卷", orth_curly_bristle)
print("     性别: 雌", orth_female, ", 雄", orth_male)
print("     灰红直长雌", f1_orth_female, ", 灰白焦小雄", f1_orth_male)

print("f1代反交数据")
comp_sum = 0
for group in f1_data:
    comp_sum = group.get_count("comp") + comp_sum
    
print("* 反交数据总数：", comp_sum)
print("* 反交数据各表型数据：")
comp_grey_body = comp_black_body = 0
comp_red_eye = comp_white_eye = 0
comp_long_wing = comp_short_wing = 0
comp_straight_bristle = comp_curly_bristle = 0
comp_female = comp_male = 0
f1_comp_female = f1_comp_male = 0 # 灰红直长雌, 灰红直长雄

for group in f1_data:
    comp_grey_body = comp_grey_body + group.get_body_color_count("grey", "comp")
    comp_black_body = comp_black_body + group.get_body_color_count("black", "comp")
    comp_red_eye = comp_red_eye + group.get_eye_color_count("red", "comp")
    comp_white_eye = comp_white_eye + group.get_eye_color_count("white", "comp")
    comp_long_wing = comp_long_wing + group.get_wing_shape_count("long", "comp")
    comp_short_wing = comp_short_wing + group.get_wing_shape_count("short", "comp")
    comp_straight_bristle = comp_straight_bristle + group.get_bristle_type_count("straight", "comp")
    comp_curly_bristle = comp_curly_bristle + group.get_bristle_type_count("curly", "comp")
    comp_female = comp_female + group.get_gender_count("female", "comp")
    comp_male = comp_male + group.get_gender_count("male", "comp")
    f1_comp_male = f1_comp_male + group.get_specific_count([1, 1, 1, 1, 0], "comp")
    f1_comp_female = f1_comp_female + group.get_specific_count([1, 1, 1, 1, 1], "comp")
    
print("     体色: 灰", comp_grey_body, ", 黑", comp_black_body)
print("     眼色: 红", comp_red_eye, ", 白", comp_white_eye)
print("     翅型: 长", comp_long_wing, ", 短", comp_short_wing)
print("     刚毛: 直", comp_straight_bristle, ", 卷", comp_curly_bristle)
print("     性别: 雌", comp_female, ", 雄", comp_male)
print("     灰红长直雌", f1_comp_female, ", 灰红长直雄", f1_comp_male)

f1代正交数据
* 正交数据总数： 2973
* 正交数据各表型数据：
     体色: 灰 2973 , 黑 0
     眼色: 红 1618 , 白 1355
     翅型: 长 1618 , 短 1355
     刚毛: 直 1618 , 卷 1355
     性别: 雌 1618 , 雄 1355
     灰红直长雌 1618 , 灰白焦小雄 1355
f1代反交数据
* 反交数据总数： 6379
* 反交数据各表型数据：
     体色: 灰 6379 , 黑 0
     眼色: 红 6379 , 白 0
     翅型: 长 6379 , 短 0
     刚毛: 直 6379 , 卷 0
     性别: 雌 3223 , 雄 3156
     灰红长直雌 3223 , 灰红长直雄 3156


* 卡平方检验

In [6]:
observed_data = np.array([f1_orth_female, f1_orth_male])
expected_data = np.array([orth_sum * 0.5, orth_sum * 0.5])
chi2, p, dof, expected = chi2_contingency([observed_data, expected_data])
print("f1正交卡方检验结果：")
print("    卡方值：", chi2)
print("    p值：", )
print("    自由度：", dof)

observed_data = np.array([f1_comp_female, f1_comp_male])
expected_data = np.array([comp_sum * 0.5, comp_sum * 0.5])
chi2, p, dof, expected = chi2_contingency([observed_data, expected_data])
print("f1反交卡方检验结果：")
print("    卡方值：", chi2)
print("    p值：", p)
print("    自由度：", dof)

f1正交卡方检验结果：
    卡方值： 11.479067341269548
    p值：
    自由度： 1
f1反交卡方检验结果：
    卡方值： 0.33117389293543964
    p值： 0.5649686736181281
    自由度： 1


2.F2代中正交反交分别都要有以下数据分析：
①基因的分离定律：选择体色，计算和验证（注：正反交结果应相似）；

* 获取体色数据

In [7]:
orth_grey_body = orth_black_body = 0
comp_grey_body = comp_black_body = 0
orth_sum = comp_sum = 0
for group in f2_data:
    orth_grey_body = orth_grey_body + group.get_body_color_count("grey", "orth")
    orth_black_body = orth_black_body + group.get_body_color_count("black", "orth")
    comp_grey_body = comp_grey_body + group.get_body_color_count("grey", "comp")
    comp_black_body = comp_black_body + group.get_body_color_count("black", "comp")
    orth_sum = orth_sum + group.get_count("orth")
    comp_sum = comp_sum + group.get_count("comp")

print("f2总数据：")
print("    正交总数：", orth_sum)
print("    反交总数：", comp_sum)
print("f2正交体色数据：")
print("    灰", orth_grey_body, ", 黑", orth_black_body)
print("f2反交体色数据：")
print("    灰", comp_grey_body, ", 黑", comp_black_body)

f2总数据：
    正交总数： 8958
    反交总数： 4716
f2正交体色数据：
    灰 6880 , 黑 2078
f2反交体色数据：
    灰 3554 , 黑 1162


* 卡平方检验

In [8]:
observed_orth_data = np.array([orth_grey_body, orth_black_body])
expected_data = np.array([orth_sum * 0.75, orth_sum * 0.25])
chi2, p, dof, expected = chi2_contingency([observed_orth_data, expected_data])
print("f2正交卡方检验结果：")
print("    卡方值：", chi2)
print("    p值：", p)
print("    自由度：", dof)

observed_comp_data = np.array([comp_grey_body, comp_black_body])
expected_data = np.array([comp_sum * 0.75, comp_sum * 0.25])
chi2, p, dof, expected = chi2_contingency([observed_comp_data, expected_data])
print("f2反交卡方检验结果：")
print("    卡方值：", chi2)
print("    p值：", p)
print("    自由度：", dof)

chi2, p, dof, expected = chi2_contingency([observed_orth_data, observed_comp_data])
print("f2正反交比较卡方检验结果：")
print("    卡方值：", chi2)
print("    p值：", p)
print("    自由度：", dof)


f2正交卡方检验结果：
    卡方值： 7.860818618252946
    p值： 0.005051751529436341
    自由度： 1
f2反交卡方检验结果：
    卡方值： 0.14545707776087888
    p值： 0.7029151001828229
    自由度： 1
f2正反交比较卡方检验结果：
    卡方值： 3.475720447548735
    p值： 0.062275598497870295
    自由度： 1


In [9]:
#TODO: 画图
#df = pd.DataFrame([
#    [orth_grey_body], [orth_black_body],
#    [comp_grey_body], [comp_black_body]
#], columns = ["grey_orth", "black_orth", "grey_comp", "black_comp"])
#
#print(df)
#
#plt.rcParams['font.family'] = ['Times New Roman']
#plt.rcParams["axes.labelsize"] = 18
#palette=['#0073C2FF','#EFC000FF']
#
#fig,ax = plt.subplots(figsize=(5,4),dpi=100,facecolor="w")
#ax = (sns.barplot(data=df))
#                #,palette)=palette,
#                # estimator=np.mean,ci="sd", capsize=.1,#errwidth=1,errcolor="k",
#                # ax=ax,
#                 #**{"edgecolor":"k","linewidth":1})

②基因的自由组合定律：选择体色和另外三种（眼色、翅形、刚毛）中任一种，计算和验证（注：正反交结果应不同）；

In [10]:
#  检测自由组合函数
def f2_check_free_combination(trait_list, type):
    count_1 = count_2 = count_3 = count_4 = 0
    for group in f2_data:
        count_1 = count_1 + group.get_motify_specific_count(trait_list, [1,1], type)
        count_2 = count_2 + group.get_motify_specific_count(trait_list, [1,0], type)
        count_3 = count_3 + group.get_motify_specific_count(trait_list, [0,1], type)
        count_4 = count_4 + group.get_motify_specific_count(trait_list, [0,0], type)
    print(count_1, count_2, count_3, count_4)
    sum = count_1 + count_2 + count_3 + count_4
    observed_data = np.array([count_1, count_2, count_3, count_4])
    if type == "orth":
        expected_data = np.array([sum * 3 / 8, sum * 3 / 8, sum * 1 / 8, sum * 1 / 8])
    elif type == "comp":
        expected_data = np.array([sum * 9 / 16, sum * 3 / 16, sum * 3 / 16, sum * 1 / 16])
    else:
        print("type error")
        return
    chi2, p, dof, expected = chi2_contingency([observed_data, expected_data])
    print("f2", type, "卡方检验结果：")
    print("    卡方值：", chi2)
    print("    p值：", p)
    print("    自由度：", dof)

# f2 正交
######## 体色+眼色 ########
print("f2正交体色+眼色")
f2_check_free_combination([1,1,0,0,0], "orth")
######## 体色+翅形 ########
print("f2正交体色+翅形")
f2_check_free_combination([1,0,1,0,0], "orth")
######## 体色+刚毛 ########
print("f2正交体色+刚毛")
f2_check_free_combination([1,0,0,1,0], "orth")
# f2 反交
######## 体色+眼色 ########
print("f2反交体色+眼色")
f2_check_free_combination([1,1,0,0,0], "comp")
######## 体色+翅形 ########
print("f2反交体色+翅形")
f2_check_free_combination([1,0,1,0,0], "comp")
######## 体色+刚毛 ########
print("f2反交体色+刚毛")
f2_check_free_combination([1,0,0,1,0], "comp")

# 正反交比较
def orth_comp_compare(trait_list):
    count_1 = count_2 = count_3 = count_4 = 0
    for group in f2_data:
        count_1 = count_1 + group.get_motify_specific_count(trait_list, [1,1], "orth")
        count_2 = count_2 + group.get_motify_specific_count(trait_list, [1,0], "orth")
        count_3 = count_3 + group.get_motify_specific_count(trait_list, [0,1], "orth")
        count_4 = count_4 + group.get_motify_specific_count(trait_list, [0,0], "orth")
    print(count_1, count_2, count_3, count_4)
    sum = count_1 + count_2 + count_3 + count_4
    observed_orth_data = np.array([count_1, count_2, count_3, count_4])
    
    count_1 = count_2 = count_3 = count_4 = 0
    for group in f2_data:
        count_1 = count_1 + group.get_motify_specific_count(trait_list, [1,1], "comp")
        count_2 = count_2 + group.get_motify_specific_count(trait_list, [1,0], "comp")
        count_3 = count_3 + group.get_motify_specific_count(trait_list, [0,1], "comp")
        count_4 = count_4 + group.get_motify_specific_count(trait_list, [0,0], "comp")
    print(count_1, count_2, count_3, count_4)
    sum = count_1 + count_2 + count_3 + count_4
    observed_comp_data = np.array([count_1, count_2, count_3, count_4])
    
    chi2, p, dof, expected = chi2_contingency([observed_orth_data, observed_comp_data])
    print("f2正反交比较卡方检验结果：")
    print("    卡方值：", chi2)
    print("    p值：", p)
    print("    自由度：", dof)

print("f2正反交比较：")
######## 体色+眼色 ########
orth_comp_compare([1,1,0,0,0])

######## 体色+翅形 ########
orth_comp_compare([1,0,1,0,0])

######## 体色+刚毛 ########
orth_comp_compare([1,0,0,1,0])


f2正交体色+眼色
3696 3184 1154 924
f2 orth 卡方检验结果：
    卡方值： 40.03180911324566
    p值： 1.0490941038265486e-08
    自由度： 3
f2正交体色+翅形
3768 3112 1159 919
f2 orth 卡方检验结果：
    卡方值： 53.33207885517564
    p值： 1.5575649932252824e-11
    自由度： 3
f2正交体色+刚毛
3785 3095 1173 905
f2 orth 卡方检验结果：
    卡方值： 60.20447730475534
    p值： 5.315697746332513e-13
    自由度： 3
f2反交体色+眼色
2789 765 873 289
f2 comp 卡方检验结果：
    卡方值： 12.162517249272335
    p值： 0.0068466817573012975
    自由度： 3
f2反交体色+翅形
2750 804 814 348
f2 comp 卡方检验结果：
    卡方值： 12.882726587362619
    p值： 0.0048972031146906694
    自由度： 3
f2反交体色+刚毛
2706 848 847 315
f2 comp 卡方检验结果：
    卡方值： 2.7617227233250428
    p值： 0.4298398607068732
    自由度： 3
f2正反交比较：
3696 3184 1154 924
2789 765 873 289
f2正反交比较卡方检验结果：
    卡方值： 734.7538853151445
    p值： 6.106915421890216e-159
    自由度： 3
3768 3112 1159 919
2750 804 814 348
f2正反交比较卡方检验结果：
    卡方值： 576.4429655953361
    p值： 1.2884576302109192e-124
    自由度： 3
3785 3095 1173 905
2706 848 847 315
f2正反交比较卡方检验结果：
    卡方值： 533.141029409010

③基因的伴性遗传定律：选择眼色、翅形、刚毛中任一种，计算和验证（注：正反交结果应不同）；

In [11]:
def f2_check_gender_lingage(trait_list, type):
    count_1 = count_2 = count_3 = count_4 = 0
    for group in f2_data:
        count_1 = count_1 + group.get_motify_specific_count(trait_list, [1,1], type)
        count_2 = count_2 + group.get_motify_specific_count(trait_list, [1,0], type)
        count_3 = count_3 + group.get_motify_specific_count(trait_list, [0,1], type)
        count_4 = count_4 + group.get_motify_specific_count(trait_list, [0,0], type)
    print(count_1, count_2, count_3, count_4)
    sum = count_1 + count_2 + count_3 + count_4
    observed_data = np.array([count_1, count_2, count_3, count_4])
    if type == "orth":
        expected_data = np.array([sum * 0.25, sum * 0.25, sum * 0.25, sum * 0.25])
    elif type == "comp":
        expected_data = np.array([sum * 0.75, sum * 0.25, 0, 0])
    else:
        print("type error")
        return
    chi2, p, dof, expected = chi2_contingency([observed_data, expected_data])
    print("f2", type, "卡方检验结果：")
    print("    卡方值：", chi2)
    print("    p值：", p)
    print("    自由度：", dof)
    
# f2 正交

print("f2正交")
######## 眼色 ########
f2_check_free_combination([0,1,0,0,1], "orth")

######## 翅形 ########
f2_check_free_combination([0,0,1,0,1], "orth")

######## 刚毛 ########
f2_check_free_combination([0,0,0,1,1], "orth")

# f2 反交

print("f2反交")
######## 眼色 ########
f2_check_free_combination([0,1,0,0,1], "comp")

######## 翅形 ########
f2_check_free_combination([0,0,1,0,1], "comp")

######## 刚毛 ########
f2_check_free_combination([0,0,0,1,1], "comp")

# 正反交比较
def orth_comp_compare(trait_list):
    count_1 = count_2 = count_3 = count_4 = 0
    for group in f2_data:
        count_1 = count_1 + group.get_motify_specific_count(trait_list, [1,1], "orth")
        count_2 = count_2 + group.get_motify_specific_count(trait_list, [1,0], "orth")
        count_3 = count_3 + group.get_motify_specific_count(trait_list, [0,1], "orth")
        count_4 = count_4 + group.get_motify_specific_count(trait_list, [0,0], "orth")
    print(count_1, count_2, count_3, count_4)
    sum = count_1 + count_2 + count_3 + count_4
    observed_orth_data = np.array([count_1, count_2, count_3, count_4])
    
    count_1 = count_2 = count_3 = count_4 = 0
    for group in f2_data:
        count_1 = count_1 + group.get_motify_specific_count(trait_list, [1,1], "comp")
        count_2 = count_2 + group.get_motify_specific_count(trait_list, [1,0], "comp")
        count_3 = count_3 + group.get_motify_specific_count(trait_list, [0,1], "comp")
        count_4 = count_4 + group.get_motify_specific_count(trait_list, [0,0], "comp")
    print(count_1, count_2, count_3, count_4)
    sum = count_1 + count_2 + count_3 + count_4
    observed_comp_data = np.array([count_1, count_2, count_3, count_4])
    
    chi2, p, dof, expected = chi2_contingency([observed_orth_data, observed_comp_data])
    print("f2正反交比较卡方检验结果：")
    print("    卡方值：", chi2)
    print("    p值：", p)
    print("    自由度：", dof)

print("f2正反交比较：")
######## 眼色 ########
orth_comp_compare([0,1,0,0,1])

######## 翅形 ########
orth_comp_compare([0,0,1,0,1])

######## 刚毛 ########
orth_comp_compare([0,0,0,1,1])

#TODO: 画图

f2正交
2725 2125 2197 1911
f2 orth 卡方检验结果：
    卡方值： 900.3451975992647
    p值： 7.449902488242646e-195
    自由度： 3
2775 2152 2147 1884
f2 orth 卡方检验结果：
    卡方值： 837.5715348583985
    p值： 3.073411506383234e-181
    自由度： 3
2787 2171 2135 1865
f2 orth 卡方检验结果：
    卡方值： 811.3555007126258
    p值： 1.49097900294701e-175
    自由度： 3
f2反交
2629 1033 174 880
f2 comp 卡方检验结果：
    卡方值： 779.9019068356743
    p值： 9.884815350472852e-169
    自由度： 3
2554 1010 249 903
f2 comp 卡方检验结果：
    卡方值： 675.199827298313
    p值： 5.006317855322292e-146
    自由度： 3
2551 1002 252 911
f2 comp 卡方检验结果：
    卡方值： 676.1074354662653
    p值： 3.182179293718239e-146
    自由度： 3
f2正反交比较：
2725 2125 2197 1911
2629 1033 174 880
f2正反交比较卡方检验结果：
    卡方值： 1294.9032708561763
    p值： 1.8781105821410635e-280
    自由度： 3
2775 2152 2147 1884
2554 1010 249 903
f2正反交比较卡方检验结果：
    卡方值： 1056.093127093375
    p值： 1.2203743704923438e-228
    自由度： 3
2787 2171 2135 1865
2551 1002 252 911
f2正反交比较卡方检验结果：
    卡方值： 1038.3459683524968
    p值： 8.641051121750097e-225


④基因的连锁交换定律：同时选择眼色、翅形、刚毛数据进行计算单交换、双交换、遗传做图（基因位置和图距）、并发系数和干涉，只计算、画图、不验证。

In [12]:
bin_list = list(product([0,1], repeat = 4))
print("\t orth\t comp")
for bin in bin_list:
    count_orth = count_comp = 0
    for item in f2_data:
        count_orth = count_orth + item.get_motify_specific_count([0,1,1,1,1],bin,"orth")
        count_comp = count_comp + item.get_motify_specific_count([0,1,1,1,1],bin,"comp")
    if bin[0] == 0:
        print("白", end = "")
    else:
        print("红", end = "")
    if bin[1] == 0:
        print("小", end = "")
    else:
        print("长", end = "")
    if bin[2] == 0:
        print("焦", end = "")
    else:
        print("直", end = "")
    if bin[3] == 0:
        print("雄", end = "")
    else:
        print("雌", end = "")
    print("\t",count_orth, "\t", count_comp)


	 orth	 comp
白小焦雄	 1218 	 583
白小焦雌	 1393 	 82
白小直雄	 96 	 29
白小直雌	 116 	 24
白长焦雄	 318 	 140
白长焦雌	 366 	 42
白长直雄	 279 	 128
白长直雌	 322 	 26
红小焦雄	 235 	 137
红小焦雌	 256 	 39
红小直雄	 335 	 154
红小直雌	 382 	 104
红长焦雄	 94 	 51
红长焦雌	 120 	 89
红长直雄	 1461 	 691
红长直雌	 1967 	 2397


# f2 正交 非交换类型
# 白小焦雌, 红长直雌
# 白小焦雄, 红长直雄
f2_orth_white_short_curly_female = f2_orth_red_long_straight_female = 0
f2_orth_white_short_curly_male = f2_orth_red_long_straight_male = 0

for item in f2_data:
    f2_orth_white_short_curly_female = f2_orth_white_short_curly_female + item.get_motify_specific_count([0,1,1,1,1], [0, 0, 0, 1], "orth")
    f2_orth_red_long_straight_female = f2_orth_red_long_straight_female + item.get_motify_specific_count([0,1,1,1,1], [1, 1, 1, 1], "orth")
    f2_orth_white_short_curly_male = f2_orth_white_short_curly_male + item.get_motify_specific_count([0,1,1,1,1], [0, 0, 0, 0], "orth")
    f2_orth_red_long_straight_male = f2_orth_red_long_straight_male + item.get_motify_specific_count([0,1,1,1,1], [1, 1, 1, 0], "orth")
    
list_non_exchange_df = pd.DataFrame([
    [f2_orth_white_short_curly_female, f2_orth_red_long_straight_female],
    [f2_orth_white_short_curly_male, f2_orth_red_long_straight_male]
], columns=["白小焦", "红长直"], index=["雌", "雄"])

print("f2正交非交换类型数据：")
print(list_non_exchange_df)

# f2 正交  单交换
# 白长直雌, 红小焦雌, 红长焦雌, 白小直雌
# 白长直雄, 红小焦雄, 红长焦雄, 白小直雄

f2_orth_white_long_straight_female = f2_orth_red_short_curly_female = f2_orth_red_long_curly_female = f2_orth_white_short_straight_female = 0
f2_orth_white_long_straight_male = f2_orth_red_short_curly_male = f2_orth_red_long_curly_male = f2_orth_white_short_straight_male = 0

for item in f2_data:
    f2_orth_white_long_straight_female =  f2_orth_white_long_straight_female + item.get_motify_specific_count([0,1,1,1,1], [0, 1, 1, 1], "orth")
    f2_orth_red_short_curly_female = f2_orth_red_short_curly_female + item.get_motify_specific_count([0,1,1,1,1], [1, 0, 0, 1], "orth")
    f2_orth_red_long_curly_female = f2_orth_red_long_curly_female + item.get_motify_specific_count([0,1,1,1,1], [1, 1, 0, 1], "orth")
    f2_orth_white_short_straight_female =f2_orth_white_short_straight_female + item.get_motify_specific_count([0,1,1,1,1], [0, 0, 1, 1], "orth")
    f2_orth_white_long_straight_male = f2_orth_white_long_straight_male + item.get_motify_specific_count([0,1,1,1,1], [0, 1, 1, 0], "orth")
    f2_orth_red_short_curly_male = f2_orth_red_short_curly_male + item.get_motify_specific_count([0,1,1,1,1], [1, 0, 0, 0], "orth")
    f2_orth_red_long_curly_male = f2_orth_red_long_curly_male + item.get_motify_specific_count([0,1,1,1,1], [1, 1, 0, 0], "orth")
    f2_orth_white_short_straight_male = f2_orth_white_short_straight_male + item.get_motify_specific_count([0,1,1,1,1], [0, 0, 1, 0], "orth")
    
list_single_exchange_df = pd.DataFrame([
    [f2_orth_white_long_straight_female, f2_orth_red_short_curly_female, f2_orth_red_long_curly_female, f2_orth_white_short_straight_female], 
    [f2_orth_white_long_straight_male, f2_orth_red_short_curly_male, f2_orth_red_long_curly_male, f2_orth_white_short_straight_male]
])

print("f2正交单交换类型数据：")
print(list_single_exchange_df)

# f2 正交  双交换
# 白长焦雌, 红短直雌,
# 白长焦雄, 红短直雄,

f2_orth_white_long_curly_female = f2_orth_red_short_straight_female = 0
f2_orth_white_long_curly_male = f2_orth_red_short_straight_male = 0

for item in f2_data:
    f2_orth_white_long_curly_female = f2_orth_white_long_curly_female + item.get_motify_specific_count([0,1,1,1,1], [0, 1, 0, 1], "orth")
    f2_orth_red_short_straight_female = f2_orth_red_short_straight_female + item.get_motify_specific_count([0,1,1,1,1], [1, 0, 1, 1], "orth")
    f2_orth_white_long_curly_male = f2_orth_white_long_curly_male + item.get_motify_specific_count([0,1,1,1,1], [0, 1, 0, 0], "orth")
    f2_orth_red_short_straight_male = f2_orth_red_short_straight_male + item.get_motify_specific_count([0,1,1,1,1], [1, 0, 1, 0], "orth")

list_double_exchange_df = pd.DataFrame([
    [f2_orth_white_long_curly_female, f2_orth_red_short_straight_female], 
    [f2_orth_white_long_curly_male, f2_orth_red_short_straight_male]
])

print("f2正交双交换类型数据：")
print(list_double_exchange_df)

In [13]:
new_bin = [
    [1, 1, 1], [0, 0, 0], [0, 1, 1], [1, 0, 0],
    [1, 0, 1], [0, 1, 0], [1, 1, 0], [0, 0, 1],
]
all_list = []
sum = 0
for item in new_bin:
    for color in [1,0]:
        for sex in [1,0]:
            new_list = item.copy()
            new_list.insert(0, color)
            new_list.append(sex)
            all_list.append(new_list)
i = 0
for bin in all_list:
    count = 0
    for item in f2_data:
        count = count + item.get_specific_count(bin,"comp")
    i = i + 1
    sum = sum + count
    print(count, end="\t\t")
    if i % 4 == 0:
        print(sum,"\n")
        sum = 0

1884		506		513		185		3088 

46		422		36		161		665 

13		97		13		31		154 

21		109		18		28		176 

70		104		34		50		258 

35		120		7		20		182 

65		30		24		21		140 

12		20		12		9		53 
